In [94]:
using RCall, Random, GLMNet, InvertedIndices, Statistics, Distributions

In [95]:
nrows, ncols = 8, 5
@rput ncols
@rput nrows
R"set.seed(1234)"
X = rcopy(R"matrix(rnorm(nrows*ncols), nrows, ncols )")
X

8×5 Array{Float64,2}:
 -1.20707   -0.564452   -0.51101   -0.69372    -0.70944 
  0.277429  -0.890038   -0.911195  -1.4482     -0.501258
  1.08444   -0.477193   -0.837172   0.574756   -1.62909 
 -2.3457    -0.998386    2.41584   -1.02366    -1.16762 
  0.429125  -0.776254    0.134088  -0.0151383  -2.18004 
  0.506056   0.0644588  -0.490686  -0.935949   -1.34099 
 -0.57474    0.959494   -0.440548   1.1023     -0.294294
 -0.546632  -0.110285    0.459589  -0.475593   -0.465898

In [99]:
k = 1
fitreg = glmnet(X[:,Not(k)],X[:,k], standardize=false)

Least Squares GLMNet Solution Path (73 solutions for 4 predictors in 337 passes):
────────────────────────────────
      df    pct_dev            λ
────────────────────────────────
 [1]   0  0.0        0.806658   
 [2]   1  0.0994775  0.734997   
 [3]   1  0.182066   0.669702   
 [4]   1  0.250632   0.610207   
 [5]   1  0.307556   0.555998   
 [6]   1  0.354816   0.506605   
 [7]   1  0.394052   0.461599   
 [8]   1  0.426627   0.420592   
 [9]   1  0.45367    0.383228   
[10]   1  0.476123   0.349183   
[11]   1  0.494763   0.318162   
[12]   1  0.510238   0.289898   
[13]   2  0.547479   0.264144   
[14]   2  0.591579   0.240678   
[15]   2  0.628192   0.219297   
[16]   2  0.658588   0.199815   
[17]   2  0.683824   0.182064   
[18]   2  0.704775   0.16589    
[19]   2  0.722168   0.151153   
[20]   2  0.736609   0.137725   
[21]   2  0.748598   0.12549    
[22]   2  0.758552   0.114342   
[23]   2  0.766815   0.104184   
[24]   2  0.773676   0.0949285  
[25]   2  0.779371   0.0864

In [100]:
fitreg.betas

4×73 CompressedPredictorMatrix:
 0.0   0.0         0.0       0.0       …   0.247236    0.248887    0.250126 
 0.0  -0.0691671  -0.13219  -0.189613     -0.806585   -0.806648   -0.806707 
 0.0   0.0         0.0       0.0          -0.0900087  -0.0910518  -0.0918609
 0.0   0.0         0.0       0.0          -0.901522   -0.902627   -0.903507 

In [96]:
predict(fitreg,X[:,Not(k)])

8×73 Array{Float64,2}:
 -0.297136  -0.263356  -0.232578  …  -0.2213     -0.221642   -0.221901 
 -0.297136  -0.235677  -0.179677     -0.0987827  -0.0990805  -0.0992915
 -0.297136  -0.240797  -0.189463      0.778265    0.777781    0.777432 
 -0.297136  -0.465798  -0.619476     -2.24658    -2.24697    -2.24727  
 -0.297136  -0.307976  -0.317853      0.470709    0.470893    0.471078 
 -0.297136  -0.264762  -0.235264  …   0.508959    0.510604    0.511875 
 -0.297136  -0.26823   -0.241892     -0.43728    -0.437443   -0.437636 
 -0.297136  -0.33049   -0.360881     -1.13108    -1.13123    -1.13137  

In [97]:
function prec_xia(X)

    nrows, ncols = size(X)

    betahat = zeros(Float64,(ncols-1,ncols))
    reshat  = copy(X)

    for k in 1:ncols
      fitreg = glmnet(X[:,Not(k)],X[:,k], standardize=false) 
      betahat[:,k] .= vec(fitreg.betas)
      reshat[:,k]  .= X[:,k] .- vec(predict(fitreg,X[:,Not(k)]))
    end

    rtilde = cov(reshat) .* (n-1) ./ n
    rhat   = rtilde

    for i in 1:ncols-1
       for j in (i+1):ncols
        rhat[i,j] = -(rtilde[i,j]+rtilde[i,i]*betahat[i,j]+rtilde[j,j]*betahat[j-1,i])
        rhat[j,i] = rhat[i,j]
      end
    end

    Tprec    = copy(1 ./ rhat)
    TprecStd = copy(Tprec)

    for i in 1:(ncols-1)
        for j in (i+1):ncols
            Tprec[i,j] = Tprec[j,i] = rhat[i,j]/(rhat[i,i]*rhat[j,j])
            thetahatij = (1+(betahat[i,j]^2  * rhat[i,i]/rhat[j,j]))/(n*rhat[i,i]*rhat[j,j])   
            TprecStd[i,j] = TprecStd[j,i] = Tprec[i,j]/sqrt(thetahatij)
        end
    end
    
    Tprec, TprecStd
end

prec_xia (generic function with 1 method)

In [98]:
prec_xia(X)

DimensionMismatch: DimensionMismatch("array could not be broadcast to match destination")